# TDT4173 - Machine Learning
## Final Notebook of project


Students: TESTARD Arthur, VERDON Valentin and VERDIER Nahel

## Contents

0. Imports and variable initialization
1. Data analysis
    - Size analysis, data type, number of features, split into data categories
    - NAN analysis
    - Correlatiosn analysis 
    - #Analyse avec le reshaped

2. Research leads
    - Signal analysis 
    - Signal treatment model with filter and correlations with noises -> Prophet
    - AutoML (keras)
    - #Analyse de la linéarité entre les entrées/sortie
    - #Mettre une étape pour réguler les données simulées par rapport aux données observées 

3. Preprocessing: 
    - Columns selection
    - NANs management
    - Columns creation
    - Normalizations (StandardScaler, Normalizer, RobustScaler, MinMaxScaler)
    - Train/Test split
    - décaler les dates de sortie / entrée (si j’ai le temps diff x_t et x_t-1 pour prédire y_t)

4. Model XGBoost: 
    - Different models testing (RandomForest / LinearRegressor)
    - Hyperparameters study
    - Features importance


## 0. Imports and data initialization

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
# import networkx as nx
import scipy
import pickle
from datetime import datetime, timedelta

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
class DataFolder:
    def __init__(self, folder_name: str):
        self.folder_name: str
        self.X_test_estimated: str = f"{folder_name}/X_test_estimated.parquet"
        self.X_train_estimated: str = f"{folder_name}/X_train_estimated.parquet"
        self.X_train_observed: str = f"{folder_name}/X_train_observed.parquet"
        self.train_targets: str | None = f"{folder_name}/train_targets.parquet"

A = DataFolder(folder_name='A')
B = DataFolder(folder_name='B')
C = DataFolder(folder_name='C')

In [ ]:
def read_files(diff_path: str = ''):
    train_a = pd.read_parquet(diff_path + A.train_targets)
    train_b = pd.read_parquet(diff_path + B.train_targets)
    train_c = pd.read_parquet(diff_path + C.train_targets)

    X_train_estimated_a = pd.read_parquet(diff_path + A.X_train_estimated)
    X_train_estimated_b = pd.read_parquet(diff_path + B.X_train_estimated)
    X_train_estimated_c = pd.read_parquet(diff_path + C.X_train_estimated)

    X_train_observed_a = pd.read_parquet(diff_path + A.X_train_observed)
    X_train_observed_b = pd.read_parquet(diff_path + B.X_train_observed)
    X_train_observed_c = pd.read_parquet(diff_path + C.X_train_observed)

    X_test_estimated_a = pd.read_parquet(diff_path + A.X_test_estimated)
    X_test_estimated_b = pd.read_parquet(diff_path + B.X_test_estimated)
    X_test_estimated_c = pd.read_parquet(diff_path + C.X_test_estimated)

    return train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c

## 1. Data analysis

## 2. Research leads

### 1. Signal analysis

Because our data were presenting some periodicities, intuitively, one of our first idea were to analyse the different signals we have, starting by our target, `pv_measurement`. However, as we can see on the following plot, the data is not completly cleared, specially on B and C. We will come back to this point in Preprocessing part.

In [ ]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = read_files()

In [ ]:
def filter_dates_when_constants(df, date_c = 'time', y = 'pv_measurement', delta = { 'days': 3 }):
    df = df.copy()
    mask_y_change = df[y] != df[y].shift(1)

    start_date = None
    end_date = None

    constant_periods = []

    for index, row in df.iterrows():
        if not mask_y_change[index]:
            if start_date is None:
                start_date = row[date_c]
            end_date = row[date_c]
        else:
            if start_date is not None and (end_date - start_date) >= pd.Timedelta(**delta):
                constant_periods.append((start_date, end_date))
            start_date = None
            end_date = None

    if start_date is not None and (end_date - start_date) >= pd.Timedelta(**delta):
        constant_periods.append((start_date, end_date))
    return constant_periods

def delete_date_range_from_df(df, dates, date_c = 'time'):
    df = df.copy()
    c = 0
    for start_date, end_date in dates:
        mask = (df[date_c] >= start_date) & (df[date_c] < end_date)
        df = df[~mask]
    df.reset_index(drop=True, inplace=True)
    return df

delta = { 'hours': 12 * 4}
train_a = delete_date_range_from_df(train_a, filter_dates_when_constants(train_a, delta=delta))
train_b = delete_date_range_from_df(train_b, filter_dates_when_constants(train_b, delta=delta))
train_c = delete_date_range_from_df(train_c, filter_dates_when_constants(train_c, delta=delta))

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(30, 20), sharex=True)

train_a[['time','pv_measurement']].set_index('time').plot(ax=axs[0], title='pv_measurement on location A')
train_b[['time','pv_measurement']].set_index('time').plot(ax=axs[1], title='pv_measurement on location B')
train_c[['time','pv_measurement']].set_index('time').plot(ax=axs[2], title='pv_measurement on location C')

As we said, we then tryied to analys this signal with basic components such as Fourier analysis.

In [ ]:
def get_fft_transforms(train):
    y = train["pv_measurement"].dropna().values
    time_diff = train["time"].diff().mean().total_seconds()
    sampling_rate = 1 / time_diff

    n = len(y)
    freq = np.fft.fftfreq(n, 1 / sampling_rate)
    fft_y = np.fft.fft(y)
    amp_fft_y = np.abs(fft_y)
    phase = np.angle(fft_y)
    return freq, fft_y, amp_fft_y, phase, sampling_rate

In [ ]:
trains = { 'A': train_a.dropna(subset='pv_measurement'), 'B': train_b.dropna(subset='pv_measurement'), 'C': train_c.dropna(subset='pv_measurement') }
locations = trains.keys()

freqs, fft_ys, amp_fft_ys, phases, sampling_rates = [
    { 
        loc: get_fft_transforms(trains[loc])[k] for loc in locations 
    } 
    for k in range(5)]

In [ ]:
plt.figure(figsize=(30, 20))
k = 1
for loc in locations:
    plt.subplot(3, 1, k)
    plt.plot(freqs[loc][:len(freqs[loc])//2], amp_fft_ys[loc][:len(freqs[loc])//2])
    plt.title(f"Spectrum of the pv_measurement at {loc} location")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    plt.grid()
    k += 1
plt.show()

In [ ]:
plt.figure(figsize=(30, 20))
k = 1
for loc in locations:
    plt.subplot(3, 1, k)
    plt.plot(freqs[loc][:len(freqs[loc])//2], 20 * np.log10(amp_fft_ys[loc][:len(freqs[loc])//2]))
    plt.title(f"Spectrum in magnitude of the pv_measurement at {loc} location")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude (dB)")
    plt.grid()
    k += 1
plt.show()

We then looked on the most important frequencies in those spectrums.

In [ ]:
def print_peak_frequencies(amp_fft_y, freq, threshold, loc):
    peaks, _ = scipy.signal.find_peaks(amp_fft_y[:len(amp_fft_y)//2], height=threshold)
    peak_frequencies = freq[:len(freq)//2][peaks]

    period_size = int(1/peak_frequencies[0])
    continuous_component = np.mean(trains[loc]["pv_measurement"].dropna().values[:period_size])

    print("Location:", loc)
    print(f'Most important periods (in days): \n{1 / peak_frequencies / 3600 / 24}')
    print(f'Value of the continous component: {continuous_component}\n\n')

In [ ]:
thresholds = { 'A': .5e7, 'B': .5e6, 'C': .25e6 }
for loc in locations:
    print_peak_frequencies(amp_fft_ys[loc], freqs[loc], thresholds[loc], loc)

First commentaries:

We know from the analysis of the nan values that A got the most clean datas in term of `pv_measurement` values. So our analysis will mostly be based on what we see on A. We can notice 3 most important frequencies: one for the year, one for the day and one for a half-day (12 hours). If we look more on the frequency plot, we can notice a most little one frequency (that our threshold impeach us to read it on the last print). This seems to be a peak for a period of 8 hours, according to the code cell bellow.

Because B and C are not much clean, we can suppose that the big differencies we found with A comes from the Nan values, which create some empty cells in these frames, which are compensated by increasing the frequency values. However, we did not pay attention to it much at first be because most of our analysis were based on A data.

In [ ]:
freq_a_1 = np.min(np.where(freqs['A'] > .00003)) 
freq_a_2 = np.max(np.where(freqs['A'] < .00004))
freq_arg = np.argmax(fft_ys['A'][:len(fft_ys['A'])//2][freq_a_1:freq_a_2])
1 / freqs['A'][:len(fft_ys['A'])//2][freq_a_1:freq_a_2][freq_arg] / 3600 

We can confirm what we sayied on B and C compared to A if we look on the differents sampling rates depending on the situation. Theorically, it should be close to one hour ($=3600$ seconds) because our values are measured every hours. But if we look on `1 / sampling_rates['B']` and `1 / sampling_rates['C']` we see that it's more than it for B and C locations. This comes from Nan values and confirms our point above.

We can notice that `1 / sampling_rates['B']` is a bit bigger than an hour. We can explain it by the gap of one week between `X_train_observed_a` and `X_train_estimated_a`, which exists as well in `train_a`.

In [ ]:
1 / sampling_rates['A'] / 3600, 1 / sampling_rates['B'] / 3600, 1 / sampling_rates['C'] / 3600

The first thing we can try is to recalculate the model by the inverse of Fourier's transformation.

In [ ]:
i_fft_ys = { loc: np.fft.ifft(fft_ys[loc]) for loc in locations }

plt.figure(figsize=(30, 20))
k = 1
for loc in locations:
    plt.subplot(3, 1, k)
    plt.plot(trains[loc]['pv_measurement'], label='real pv_measurement')
    plt.plot(i_fft_ys[loc], label='ifft pv_measurement')
    plt.title(f"Spectrum of the pv_measurement at {loc} location")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    plt.grid()
    k += 1
plt.show()

We notice a gap created in C but in facts its due to the index. 

Now the idea is to keep only the most important frequencies in order to have a model which can be written like this:
$$y[n] = \hat{y}[n] + r[n]$$

where $n$ is the index of the output, $y[n]$ is the real value of `pv_measurement` at index $n$ (or time $t$), $\hat{y}[n]$ is the value at index $n$ of the signal filtered predicted by signal analysis and $r[n]$ is the value at index $n$ of the noise created by mostly, the weather, from our inputs `X_train_estimated`, `X_train_observed`, etc. It would be design by a machine learning model. Actually we did not had the time to test this feature entirely, because of a lack of time our goals priotization. So, it is not entirely designed, but we will detail as far as we came to it.

In [ ]:
end_dates = { 'A': '2022-10-21', 'B': '2022-03-15', 'C': '2022-04-01'}
# end_dates = { 'A': X_train_observed_a['date_forecast'].max(), 'B': X_train_observed_b['date_forecast'].max(), 'C': X_train_observed_c['date_forecast'].max() }

start_dates = { 'A': '2020-10-21', 'B': '2020-03-15', 'C': '2020-04-01' }

In [ ]:
def get_model(fft_values, threshold=60, sample_rate=1):

    n = len(fft_values)

    frequencies = np.fft.fftfreq(n, 1 / sample_rate)
    amplitudes = fft_values * (np.abs(fft_values) > threshold)
    phases = np.angle(fft_values)
    return {"frequencies": frequencies, "amplitudes": amplitudes, "phases": phases}

In [ ]:
def reconstruct_signal(model, duration,sample_rate):
    frequencies = model["frequencies"]
    amplitudes = model["amplitudes"]
    phases = model["phases"]

    t = np.arange(0, duration, 1)
    signal = np.zeros(len(t), dtype=np.complex128)
    
    for freq, amp, phase in tqdm(zip(frequencies, amplitudes, phases)):
        signal += amp * np.exp(2j * np.pi * freq * t / sample_rate + phase)
    return signal / len(frequencies)

In [ ]:
def get_thresholds_to_get_n_freq(signal, nb_freq, threshold, step):
    assert step > 0
    fft = np.fft.fft(signal)
    abs_fft = np.abs(fft[:len(fft)//2])

    freqs = [ f for f in abs_fft if f > threshold ]
    threshold += step
    while len(freqs) > nb_freq:
        freqs = [ f for f in abs_fft if f > threshold ]
        threshold += step
    threshold = threshold if len(freqs) > 0 else threshold - step
    return threshold

In [ ]:
def get_filtred_signal(signal, nb_freqs, sample_rate, nb_days_to_predict = 0, threshold = 0, scaler = StandardScaler):
    scaler_pred = scaler
    scaler = scaler()
    Y_normed = scaler.fit_transform(np.array(signal['pv_measurement'].dropna()).reshape(-1, 1)).reshape(-1)

    threshold = get_thresholds_to_get_n_freq(signal=Y_normed, nb_freq=nb_freqs, threshold=0, step=.5)
    model = get_model(fft_values=np.fft.fft(Y_normed), threshold=threshold, sample_rate=sample_rate)
    pred_from_model_data = np.real(reconstruct_signal(model, duration=len(model["frequencies"]) + nb_days_to_predict, sample_rate=sample_rate)) 
    scaler_pred = scaler_pred()
    pred_normed = scaler_pred.fit_transform(pred_from_model_data.reshape(-1, 1)).reshape(-1)

    Y_filtred = scaler.inverse_transform(pred_normed.reshape(-1, 1)).reshape(-1)

    # If we want to filter negative values
    Y_filtred[Y_filtred < 0] = 0
    return Y_filtred

In [ ]:
nb_freqs = 2
nb_days_to_predict = 0

trains_on_dates = { loc: trains[loc][(trains[loc]["time"] < pd.Timestamp(end_dates[loc])) & (trains[loc]["time"] >= pd.Timestamp(start_dates[loc]))] for loc in locations }
Y_filtred = { loc: get_filtred_signal(trains_on_dates[loc], nb_freqs, sampling_rates[loc], nb_days_to_predict=nb_days_to_predict) for loc in locations }

In [ ]:
sp = 1

plt.figure(figsize=(50, 30))
for loc in locations:
    plt.subplot(3, 1, sp)
    plt.plot(Y_filtred[loc], color='b')
    plt.plot(np.array(trains_on_dates[loc]['pv_measurement']), color='orange')
    plt.title(f"Prophet on location {loc}")
    sp += 1

However, we explored different ways to design $\hat{y}[n]$. The first one is a raw filter on the whole signal. This method were not much efficient. In our researchs we found `prophet`, a Python (and R) library which gives a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

In [ ]:
end_dates = { 'A': X_train_estimated_a['date_forecast'].max(), 'B': X_train_estimated_b['date_forecast'].max(), 'C': X_train_estimated_c['date_forecast'].max() }
trains = { loc: trains[loc][trains[loc]['time'] <= end_dates[loc]] for loc in locations}

prophet_scalers = { loc: MinMaxScaler() for loc in locations }

In [ ]:
trains_raw_prophet = { loc: trains[loc].dropna(subset='pv_measurement').reset_index().rename(columns={'time': 'ds', 'pv_measurement': 'y'}) for loc in locations }

trains_prophet = trains_raw_prophet
for loc in locations:
    trains_prophet[loc]['y'] = prophet_scalers[loc].fit_transform(np.array(trains_raw_prophet[loc]['y']).reshape(-1, 1)).reshape(-1)

models_prophet = { loc: Prophet(changepoint_prior_scale=0.05) for loc in locations }
predictions_prophet = {}
forecast = {}

for loc in locations:
    models_prophet[loc].fit(trains_prophet[loc])
    predictions_prophet[loc] = models_prophet[loc].make_future_dataframe(periods=66, freq='h')
    forecast[loc] = models_prophet[loc].predict(predictions_prophet[loc])

In [ ]:
sp = 1

plt.figure(figsize=(50, 30))
for loc in locations:
    plt.subplot(3, 1, sp)
    plt.plot(trains_prophet[loc]['y'], color='orange')
    plt.plot(forecast[loc]['yhat'], color='b')
    plt.title(f"Prophet on location {loc}")
    sp += 1

This results, from prophet and the filter signal, are not really satisfying. Then came the idea, inpired by [this paper](https://peerj.com/preprints/3190.pdf), to see what's happen if we plot one signal for each hour (it would make $24 * 3 = 72$ models). We then first split our signals by hours and plot what we get with prophet prediction and our filter.

In [ ]:
# USEFULL VALUES TO COMPILE

hours = [ f"0{h}" if h < 10 else str(h) for h in range(24) ]

# end_dates = { 'A': '2022-10-21', 'B': '2022-03-15', 'C': '2022-04-01'}
# start_dates = { 'A': '2020-10-21', 'B': '2020-03-15', 'C': '2020-04-01' }
nb_freqs = 1

In [ ]:
trains_on_dates = { loc: trains[loc][(trains[loc]["time"] < pd.Timestamp(end_dates[loc])) & (trains[loc]["time"] >= pd.Timestamp(start_dates[loc]))] for loc in locations }
trains_on_hours = { loc: { h: trains_on_dates[loc][trains_on_dates[loc]['time'].dt.strftime('%H:%M:%S').str.endswith(f'{h}:00:00')] for h in hours } for loc in locations }

# MAYBE DROP NA ON B AND C


In [ ]:
Y_h_filtred = { loc: { h: get_filtred_signal(trains_on_hours[loc][h], nb_freqs, sampling_rates[loc] * 24, nb_days_to_predict=nb_days_to_predict, scaler=StandardScaler) for h in hours} for loc in locations }

In [ ]:
for loc in locations:
    sp = 1
    plt.figure(figsize=(50, 30))
    for h in hours:
        plt.subplot(4, 6, sp)
        plt.plot(np.array(trains_on_hours[loc][h]['pv_measurement']), color='orange')
        plt.plot(Y_h_filtred[loc][h], color='b')
        plt.title(f"Filtred signal at time h={h} for location {loc}")
        sp += 1

In [ ]:
def convert_hours_to_days(signal):
    min_len = np.min([len(signal[h]) for h in hours ])
    y_pred = []
    for d in range(min_len):
        for h in hours:
            y_pred.append(signal[h][d])
    return np.array(y_pred)

In [ ]:
Y_h_filtred[loc]['21'].shape

In [ ]:
reconstructed_train = { loc: convert_hours_to_days(Y_h_filtred[loc]) for loc in locations }

In [ ]:
sp = 1

plt.figure(figsize=(30, 20))
for loc in locations:
    plt.subplot(3, 1, sp)
    plt.plot(reconstructed_train[loc], color='b', label='reconstruted from signal analysis')
    plt.plot(np.array(trains_on_dates[loc]['pv_measurement']), color='orange', label='real value')
    plt.title(f"Prophet on location {loc}")
    plt.legend()
    sp += 1

We get here a far more satisfying result. There is a problem for B, we did not get why the curve does not go to 0 value.

In [ ]:
df = { loc: {} for loc in locations }
trains_prophet_on_hours = { loc: { h: trains_prophet[loc][trains_prophet[loc]['ds'].dt.strftime('%H:%M:%S').str.endswith(f'{h}:00:00')] for h in hours } for loc in locations }

for loc in locations:
    for h in hours:
        date_index = [ (pd.Timestamp(start_dates[loc]) + d).strftime("%Y-%m-%d") for d in [ timedelta(days=k) for k in range(len(trains_prophet_on_hours[loc][h])) ] ]
        trains_prophet_on_hours[loc][h] = pd.DataFrame({'ds': date_index, 'y': trains_prophet_on_hours[loc][h]['y']})


In [ ]:
models_prophet = { loc: { h: Prophet(changepoint_prior_scale=0.05) for h in hours} for loc in locations }
predictions_prophet = { loc: {} for loc in locations }
forecast = { loc: {} for loc in locations }

for loc in locations:
    for h in hours:
        models_prophet[loc][h].fit(trains_prophet_on_hours[loc][h])
        predictions_prophet[loc][h] = models_prophet[loc][h].make_future_dataframe(periods=0, freq='h')
        forecast[loc][h] = models_prophet[loc][h].predict(predictions_prophet[loc][h])

In [ ]:
for loc in locations:
    sp = 1
    plt.figure(figsize=(50, 30))
    for h in hours:
        plt.subplot(4, 6, sp)
        plt.plot(np.array(trains_prophet_on_hours[loc][h].reset_index()['y']), color='orange')
        plt.plot(np.array(forecast[loc][h]['yhat']), color='b')
        plt.title(f"Prophet on location {loc}")
        sp += 1

In [ ]:
# Y_to_reconstruct = { loc: { h: prophet_scalers[loc].inverse_transform(np.array(forecast[loc][h]['yhat']).reshape(-1, 1)).reshape(-1) for h in hours } for loc in locations }
Y_to_reconstruct = { loc: { h: np.array(forecast[loc][h]['yhat']) for h in hours } for loc in locations }
reconstructed_train_prophet = { loc: convert_hours_to_days(Y_to_reconstruct[loc]) for loc in locations }


In [ ]:
sp = 1

plt.figure(figsize=(30, 20))
for loc in locations:
    plt.subplot(3, 1, sp)
    plt.plot(reconstructed_train_prophet[loc], color='b', label='reconstruted from signal analysis')
    plt.plot(np.array(trains_prophet[loc]['y']), color='orange', label='real value')
    plt.title(f"Prophet on location {loc}")
    plt.legend()
    sp += 1

These results are less satisfying than the precedent result. But let's see what happens if we try to predict the noise, $r[n]$, with Prophet.

In [ ]:
noise_prophet = { loc: { h: np.array(trains_on_hours[loc][h]['pv_measurement']) - Y_h_filtred[loc][h] for h in hours } for loc in locations }

In [ ]:
noise_prophet = { loc: { h: np.array(trains_on_hours[loc][h]['pv_measurement']) - Y_h_filtred[loc][h] for h in hours } for loc in locations }


In [ ]:
trains_noise_prophet_on_hours = { loc: {} for loc in locations }

for loc in locations:
    for h in hours:
        date_index = [ (pd.Timestamp(start_dates[loc]) + d).strftime("%Y-%m-%d") for d in [ timedelta(days=k) for k in range(len(noise_prophet[loc][h])) ] ]
        trains_noise_prophet_on_hours[loc][h] = pd.DataFrame({'ds': date_index, 'y': noise_prophet[loc][h]})

models_prophet = { loc: { h: Prophet(changepoint_prior_scale=0.05) for h in hours} for loc in locations }
predictions_prophet = { loc: {} for loc in locations }
forecast = { loc: {} for loc in locations }

for loc in locations:
    for h in hours:
        models_prophet[loc][h].fit(trains_noise_prophet_on_hours[loc][h])
        predictions_prophet[loc][h] = models_prophet[loc][h].make_future_dataframe(periods=0, freq='h')
        forecast[loc][h] = models_prophet[loc][h].predict(predictions_prophet[loc][h])

In [ ]:
for loc in locations:
    sp = 1
    plt.figure(figsize=(50, 30))
    for h in hours:
        plt.subplot(4, 6, sp)
        plt.plot(np.array(noise_prophet[loc][h]), color='orange')
        plt.plot(np.array(forecast[loc][h]['yhat']), color='b')
        plt.title(f"Prophet on location {loc}")
        sp += 1

## 3. Preprocessing

During our study, we explored many ways to pre-process our inputs. Some were compatible to each other, some were not. We are going to present in this part, all we did as pre-process and those we used.

### 1.

### 2. 

### 3. 

### 4. Interpolation of the output

One of our idea, which worked pretty well, was to interpolate the values of the output. 

### 5. Input reshaping

## 4. Model XGBoost